# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [143]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [144]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities_IR.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.72,68,100,8.80,SH,1690316619
1,1,tazovsky,67.4667,78.7000,13.05,76,38,4.66,RU,1690316619
2,2,cill airne,52.0500,-9.5167,14.49,89,40,1.80,IE,1690316619
3,3,puerto natales,-51.7236,-72.4875,0.34,79,8,2.26,CL,1690316498
4,4,grytviken,-54.2811,-36.5092,-3.64,80,67,4.93,GS,1690316498


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [161]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = 'CartoLight',
    frame_width = 1000,
    frame_height = 1000,
    # size = 'Humidity'
    # the size is commented out because there cities with very high humidity values (up to 100) that produces a ValueError 
    # ValueError [Call holoviews.ipython.show_traceback() for details]
    # Screen sizes must be positive
)
    

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [60]:
# Narrow down cities that fit criteria and drop any results with null values
# 15-26.99 temp
# 40-60 cloud
temp = f"{city_data_df['Max Temp']} > 15 & {city_data_df['Max Temp']} < 26.99"
cloud = f"{city_data_df['Cloudiness']} > 40 & {city_data_df['Cloudiness']} < 60"

ideal_weather = city_data_df.loc[(city_data_df['Max Temp'] > 15) &
                                 (city_data_df['Max Temp'] < 26.99) &
                                 (city_data_df['Cloudiness'] > 40) &
                                 (city_data_df['Cloudiness'] < 60)
                                ]

# Drop any rows with null values
ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,inongo,-1.9500,18.2667,23.26,67,58,2.63,CD,1690316620
119,119,cova figueira,14.8905,-24.2934,22.79,89,45,5.98,CV,1690316636
173,173,vila franca do campo,37.7167,-25.4333,23.23,74,49,1.23,PT,1690316643
230,230,betio village,1.3580,172.9211,26.02,74,54,2.46,KI,1690316654
235,235,celendin,-6.8703,-78.1517,23.29,34,52,2.86,PE,1690316655
261,261,beberibe,-4.1797,-38.1306,26.66,73,59,5.45,BR,1690316659
287,287,kresek,-6.1314,106.3797,26.73,86,55,1.84,ID,1690316662
304,304,kone,-21.0595,164.8658,16.82,79,41,3.70,NC,1690316665
331,331,constantia,44.1833,28.6500,25.39,69,49,9.82,RO,1690316669
334,334,dej,47.1500,23.8667,20.51,89,55,2.10,RO,1690316670


### Step 3: Create a new DataFrame called `hotel_df`.

In [87]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,inongo,CD,-1.9500,18.2667,67,
119,cova figueira,CV,14.8905,-24.2934,89,
173,vila franca do campo,PT,37.7167,-25.4333,74,
230,betio village,KI,1.3580,172.9211,74,
235,celendin,PE,-6.8703,-78.1517,34,
261,beberibe,BR,-4.1797,-38.1306,73,
287,kresek,ID,-6.1314,106.3797,86,
304,kone,NC,-21.0595,164.8658,79,
331,constantia,RO,44.1833,28.6500,69,
334,dej,RO,47.1500,23.8667,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [88]:
# Set parameters to search for a hotel
radius = 10000

params = {
    'filter': '',
    'bias': '',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, 'Lat']
    lng = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
inongo - nearest hotel: No hotel found
cova figueira - nearest hotel: No hotel found
vila franca do campo - nearest hotel: No hotel found
betio village - nearest hotel: No hotel found
celendin - nearest hotel: No hotel found
beberibe - nearest hotel: No hotel found
kresek - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
dej - nearest hotel: No hotel found
antofagasta - nearest hotel: No hotel found
mareeba - nearest hotel: No hotel found
tyumentsevo - nearest hotel: No hotel found
toliara - nearest hotel: No hotel found
saint-philippe - nearest hotel: No hotel found
la foa - nearest hotel: No hotel found
andovoranto - nearest hotel: No hotel found
bhamo - nearest hotel: No hotel found
el alto - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
13,inongo,CD,-1.9500,18.2667,67,No hotel found
119,cova figueira,CV,14.8905,-24.2934,89,No hotel found
173,vila franca do campo,PT,37.7167,-25.4333,74,No hotel found
230,betio village,KI,1.3580,172.9211,74,No hotel found
235,celendin,PE,-6.8703,-78.1517,34,No hotel found
261,beberibe,BR,-4.1797,-38.1306,73,No hotel found
287,kresek,ID,-6.1314,106.3797,86,No hotel found
304,kone,NC,-21.0595,164.8658,79,No hotel found
331,constantia,RO,44.1833,28.6500,69,No hotel found
334,dej,RO,47.1500,23.8667,89,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [162]:
%%capture --no-display

# Configure the map plot
map_plot1 = hotel_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = 'CartoLight',
    frame_width = 1000,
    frame_height = 1000,
    hover_cols = ['Hotel Name', 'City', 'Country']
)
    
# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (Hotel Name,City,Country)